In [1]:
# Системные импорты и настройки
import os
import sys
import yaml
import warnings
import ipynbname
import logging.config

warnings.filterwarnings('ignore')

# for local development
RT_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/rt_libs/src"
BA_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/basic_application/src"
sys.path.append(RT_LIBS_PATH)
sys.path.append(BA_LIBS_PATH)

In [2]:
from core import context

In [3]:
class_name = "core.context.BasicContext"
class_ = globals().get(class_name)
print(class_)

None


In [8]:
getattr(context, "BasicContext")

core.context.basic_context.BasicContext

In [6]:
import core.context

In [16]:
from core import context

In [28]:
getattr(globals()["context"], "BasicContext")

In [25]:
core.context

<module 'core.context' from '/Users/alex/Dev_projects/MyOwnRepo/rt_libs/src/core/context/__init__.py'>

In [3]:
config = {
    "context":{
        "class": "BasicContext",
        "params":{
            "market_fee": 0.0015
        },
    },
    "action_controller":{
        "class": "TickerBasic",
        "params":{ 
            "penalty": -2, 
            "reward": 0,
            "scale_wait": 10, 
            "scale_open": 10, 
            "scale_hold": 10, 
            "scale_close": 100, 
            "num_mean_obs": 2
        },
        
    },
    "observation_builder":{
        "class": "ObservationBuilder2Dim",
        "features":{
            "static":[{"class": "TradeStateFeature"}], 
            "series":[
                {"class": "Rates2DFactorFeature", "params": {"step_factor":[1, 3, 12]}},
                {"class": "ProfitFeature"},
                {"class": "OrderbookDiffFeature2D", "params": {"levels": [0.001, 0.0025, 0.005, 0.0075]}},
            ]
        }
    }
}

In [4]:
from core.facade import RTCore
from core import context
from core import tickers
from core import observation_builder
from core.observation_builder import features



class ConstructorGen1:
    """Конструктор core из конфига"""

    def get_core(self, config):
        """Основной метод для сборки core"""
        context_config = config.get("context")
        context = self._context_constructor(context_config)

        at_config = config.get("action_controller")
        action_controller = self._ticker_constructor(at_config, context)

        ob_config = config.get("observation_builder")
        observation_builder = self._observation_builder_constructor(ob_config, context)

        core = RTCore(context, action_controller, observation_builder)
        return core

    def _context_constructor(self, config):
        """Метод собирает контекст"""
        class_name = config.get("class")
        params = config.get("params")
        context = getattr(globals()["context"], class_name)(**params)
        return context

    def _ticker_constructor(self, config, context):
        """Метод собирает экшн контроллер"""
        class_name = config.get("class")
        params = config.get("params")
        action_controller = getattr(globals()["tickers"], class_name)(context, **params)
        return action_controller

    def _feature_builder(self, features, context):
        """метод собирает конкретную фичу"""
        feature_list = []
        for feature_config in features:
            feature_class_name = feature_config.get("class")
            params = feature_config.get("params")

            if params is not None:
                feat_instance = getattr(globals()["features"], feature_class_name)(context, **params)
            else:
               feat_instance = getattr(globals()["features"], feature_class_name)(context)
            feature_list.append(feat_instance)

        return feature_list

    def _observation_builder_constructor(self, config, context):
        """Метод собирает observation_builder из фичей"""
        class_name = config.get("class")

        static_config = config.get("features").get("static")
        series_config = config.get("features").get("series")

        static_feats = self._feature_builder(static_config, context)
        series_feats = self._feature_builder(series_config, context)
        
        observation_builder = getattr(globals()["observation_builder"], class_name)(context, static=static_feats, series=series_feats)
        return observation_builder

In [5]:
cg = ConstructorGen1()
rtcore = cg.get_core(config)

In [6]:
rtcore.__dict__

{'context': <core.context.basic_context.BasicContext at 0x7fd308e10850>,
 'action_controller': <core.tickers.ticker.TickerBasic at 0x7fd308e10580>,
 'observation_builder': <core.observation_builder.v5_universal.ObservationBuilder2Dim at 0x7fd308e107c0>,
 'metric_collector': <core.metrics.MetricCollector at 0x7fd308e10730>}

In [32]:
!jt -t gruvboxd -T -N -kl

In [31]:
!jt -l

Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl
